In [1]:
import numpy as np
import time
#import numpy as np
np.set_printoptions(threshold=np.inf)

def count_zeros_between_ones(binary_string):
    # Counts the number of 0's sandwiched between 1 1's
    binary_list = list(binary_string)
    bits = len(binary_list)
    count = 0
    for i in range(bits):
        if (
            (i > 0 and i < (bits - 1) and binary_list[i] == '0' and binary_list[i + 1] == '1' and binary_list[i - 1] == '1')
            or (i == 0 and binary_list[i] == '0' and binary_list[i + 1] == '1' and binary_list[bits - 1] == '1')
            or (i == (bits - 1) and binary_list[i] == '0' and binary_list[i - 1] == '1' and binary_list[0] == '1')
        ):
            count += 1
    return count


def generate_binary_arrays(num_bits):
    # Generates arrays without nearest-neighbors and evaluates the number of ones and the number of 101

    binary_arrays = []
    num_ones_counts = []
    next_to_nearest_counts = []

    for num in range(2 ** num_bits):
        binary = format(num, f'0{num_bits}b')
        binary_list = list(binary)
        bits = len(binary_list)
        valid_states = True
        for i in range(len(binary_list)):
            if (
                (i > 0 and i < (bits - 1) and binary_list[i] == '1' and ((binary_list[i + 1] == '1' or binary_list[i - 1] == '1')))
                or (i == 0 and binary_list[i] == '1' and (binary_list[i + 1] == '1' or binary_list[bits - 1] == '1'))
                or (i == (bits - 1) and binary_list[i] == '1' and (binary_list[i - 1] == '1' or binary_list[0] == '1'))
            ):
                valid_states = False
                break
        if valid_states:
            binary_arrays.append(binary)
            num_ones_counts.append(binary.count('1'))
            next_to_nearest_counts.append(count_zeros_between_ones(binary))

    # Create subsets based on the number of 1s
    subsets = {}
    for binary, num_ones, next_to_nearest in zip(binary_arrays, num_ones_counts, next_to_nearest_counts):
        subsets.setdefault(num_ones, []).append((binary, num_ones, next_to_nearest))

    # Split subsets based on the number of pairs of next-to-nearest-neighbor 1s
    split_subsets = {}
    for num_ones, subset in subsets.items():
        split_subsets[num_ones] = {}
        for binary, num_ones, next_to_nearest in subset:
            split_subsets[num_ones].setdefault(next_to_nearest, []).append(binary)

    return split_subsets


def get_ones_positions(binary_array):
    # Gets the positions of 1's
    positions = []
    for i in range(len(binary_array)):
        if binary_array[i] == '1':
            positions.append(i + 1)
    return positions


def check_link(sub, arr):
    # Checks whether a state with n flips is in a state with n+1 flips (needed for the links)
    # By construction, we just need to check that one of the elements sub is at least in one element of arr.
    # This speeds up the calculations
    for item in sub:
        if any(np.isin(item, arr[0])):
            return True
    return False


def form_matrix(num_bits, lam):
    # Forms the matrix of the equations
    # Comment the print statements if you don't need to print the form of the states

    split_subsets = generate_binary_arrays(num_bits)

    # Formation of the state
    state = 0
    state_vector = []
    state_length = []
    state_ones = []
    state_flippable = []

    flip_item = np.zeros(num_bits // 2 + 1, dtype='int')
    flip = {num_ones: {} for num_ones in range(num_bits // 2 + 1)}

    for num_ones, subset in split_subsets.items():
        for next_to_nearest, binaries in subset.items():
            state_vector.append([get_ones_positions(b) for b in binaries])
            state_length.append(len(binaries))
            state_ones.append(num_ones)
            state_flippable.append(num_bits - (num_ones * 2 - next_to_nearest))
            flip[num_ones][flip_item[num_ones]] = state
            flip_item[num_ones] += 1
            state += 1

    # Formation of the links
    links = {}
    coupling = {}

    for st in range(state):
        ind = 0
        links[st] = {}
        coupling[st] = {}
        num_ones = state_ones[st]
        if num_ones < num_bits // 2:
            for it in flip[num_ones + 1]:
                st2 = flip[num_ones + 1][it]
                if num_ones==0 or check_link(state_vector[st], state_vector[st2]):
                    links[st][ind] = st2
                    coupling[st][ind] = state_length[st2] / state_length[st] * state_ones[st2] ** 2
                    ind += 1

    mat = np.zeros((state, state))
    for st in range(state):
        mat[st][st] = state_flippable[st] * lam
        for st2 in range(len(links[st])):
            mat[st, links[st][st2]] = np.sqrt(coupling[st][st2])
            mat[links[st][st2], st] = np.sqrt(coupling[st][st2])

    return mat


# Matrix for a given number of plaquettes (num_bits) and a given value of lambda
time0 = time.time()
Mat = form_matrix(num_bits=, lam=1)
print(f"Number of equations = {Mat.shape[0]}")
print(f"Time required = {time.time() - time0}")

#print(Mat)


Number of equations = 3
Time required = 0.0
